# Rebate Email Project v.2

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  

### Define Rep
- Full name, UPPER CASE

In [ ]:
rep = input()

In [ ]:
# test data:  211122_rebate_data.csv
data = input()

In [ ]:
data = pd.read_csv(data, header = 0)
data.info()

In [ ]:
data.head()

In [ ]:
# Drop the "TOTAL" line
data.drop(0, axis = 0, inplace=True)
data = data.reset_index(drop=True)
data.head(0)

In [ ]:
def string_to_float(array):
    array = (array.str.replace("+", "", regex = False).
                   str.replace("%", "", regex = False).
                   str.replace(",", "", regex = False).
                   str.replace(":", "", regex = False).
                   replace(r"^\s*$", 0, regex = True).
                   str.strip().
                   astype(float)
            )
    return pd.Series(array)
#  replace(r"^\s*$", 0, regex = True).
cols_to_clean = ['INVOICES & PROFORMA UNITS - CY', 'INVOICES & PROFORMA NET AMOUNT  - CY']

data[cols_to_clean] = data[cols_to_clean].apply(string_to_float)

In [ ]:
data.head()

In [ ]:
data = data.rename(columns = {'Main Account: Code': 'code', 'Main Account: Name': 'customer',
                          'Main Account: City': 'city', 'Year': 'year', 'Quarter': 'quarter',
                          'INVOICES & PROFORMA UNITS - CY': 'units', 
                          'INVOICES & PROFORMA NET AMOUNT  - CY': 'dollars'})
data.head(50)

In [ ]:
data['code'] = data['code'].astype(int)

In [ ]:
data = data[['customer', 'code', 'city', 'year', 'quarter', 'units', 'dollars']]

In [ ]:
pdf = data.pivot(index=['code', 'customer', 'city'], columns=['year', 'quarter'], values=['units', 'dollars'])

pdf.head(4)

In [ ]:
def collapse_columns(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.to_series().apply(lambda x: "_".join(x))
    return df

df = collapse_columns(pdf)

df = df.reset_index()
df

In [ ]:
df.columns

In [ ]:
# Find All Rows with NA values
null_rows = df[df.isna().any(axis = 1)].head()
null_rows

In [ ]:
# Check to make sure their are no NAN's in 'Code', 'Customer', or 'City'
assert df['code'].isna().any() == 0
assert df['customer'].isna().any() == 0
assert df['city'].isna().any() == 0

# Replace NAN with 0
df = df.fillna(0)
assert df.isna().any().any() == 0
df.isna().all()

In [ ]:
# Remove customers without values in '2021_q4_dollars'
df = df[df['dollars_2021_Q4'] != 0]
#df.info()

### Import Account Contact Information

In [ ]:
cols = ["name", "code", "contact", "email"]

contact = pd.read_csv(f"account_contact_info_{rep}.csv", header = 0, usecols = [0,1,2,3], names = cols)
print("length - ", len(contact))
contact.head()

### Import Program Information

In [ ]:
columns = ["rep", "code", "customer", "vol", "vs", "el360", "level", "base_year", "static_board", "sb_doors", "tabs"]

program = pd.read_csv("quarterly_program.csv", header = 0, names = columns)
program['base_year'] = program['base_year'].astype(str).str[:4]
program = program[program['rep'] == rep]
program['code'] = program['code'].astype(int)

print("length - ", len(program))
program.head()

In [ ]:
program_color = program[['code', 'tabs']]
program_year = program[['code', 'base_year']]

# convert program DF to a dictionary
program_color_dict = program_color.set_index('code').T.to_dict('records')
program_year_dict = program_year.set_index('code').T.to_dict('records')
program_color_dict = program_color_dict[0]
#print(program_color_dict)
program_year_dict = program_year_dict[0]
#print(program_year_dict)

### CHANGE SETTINGS: Show all rows/cols of DF

In [ ]:
# # View all rows of DF
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Merge Contact Information

In [ ]:
df = pd.merge(left = df, right = contact, how = 'left', left_on = 'code', right_on = 'code')

df['color'] = df['code'].map(program_color_dict)
df['base_year'] = df['code'].map(program_year_dict)

df = df[['code', 'customer', 'city', 'contact', 'email', 'color', 'base_year',
         'units_2019_Q1', 'units_2019_Q2', 'units_2019_Q3', 'units_2019_Q4', 
         'units_2020_Q1', 'units_2020_Q2', 'units_2020_Q3', 'units_2020_Q4', 
         'units_2021_Q1', 'units_2021_Q2', 'units_2021_Q3', 'units_2021_Q4', 
         'dollars_2019_Q1', 'dollars_2019_Q2', 'dollars_2019_Q3', 'dollars_2019_Q4', 
         'dollars_2020_Q1', 'dollars_2020_Q2', 'dollars_2020_Q3', 'dollars_2020_Q4',
         'dollars_2021_Q1', 'dollars_2021_Q2', 'dollars_2021_Q3', 'dollars_2021_Q4'
        ]]

print(len(df))
df.head()

In [ ]:
# Drop any row where account's base year is 2021
df = df.drop(df[df['base_year'] == '2021'].index)

In [ ]:
# Drop any row where color or base year is NaN
null_rows = df[df.isna().any(axis = 1)]
null_rows
null_rows

## Print Reports

In [ ]:
#YES for yes, NO for no
print_reports = input()

In [ ]:
if print_reports == "YES":
    null_rows.to_excel(f'/Users/matt/Desktop/{rep}_Dropped_Accounts.xlsx', engine='xlsxwriter')

### Drop Null Rows

In [ ]:
df = df.drop(null_rows.index)
null_rows = df[df.isna().any(axis = 1)]
assert df.isna().any().any() == 0

### Set Targets

In [ ]:
aur = 80

quarterly_target = 1.12
quarterly_rebate_percentage = .06

annual_target = 1.12
annual_rebate_percentage = .08

### Quarterly Rebate Columns

In [ ]:
# Set Quarterly Target Columns
df.loc[df['base_year'] == '2019', 'Q4_target'] = df['dollars_2019_Q4'] * quarterly_target
df.loc[df['base_year'] == '2020', 'Q4_target'] = df['dollars_2020_Q4'] * quarterly_target

# Set rules for minimum target
df.loc[df['Q4_target'] < 1250, 'Q4_target'] = 1250

# Set target delta and units needed
df['Q4_target_delta'] = (df['Q4_target'] - df['dollars_2021_Q4']).astype(int)
df['Q4_units_needed'] = round((df['Q4_target_delta'] / aur)) + 1

# Calculate Quarterly Payout
df.loc[df['dollars_2021_Q4'] > df['Q4_target'], 'est_Q4_rebate_value'] = (df['dollars_2021_Q4'] * quarterly_rebate_percentage).astype(int)
df.loc[df['dollars_2021_Q4'] <= df['Q4_target'], 'est_Q4_rebate_value'] = (df['Q4_target'] * quarterly_rebate_percentage).astype(int)

# If Quarterly units needed < 0, set to zero
df.loc[df['Q4_units_needed'] < 0, 'Q4_units_needed'] = 0
df.loc[df['Q4_target_delta'] < 0, 'Q4_target_delta'] = 0

# Determine if quarterly rebate has been achieved
df.loc[df['dollars_2021_Q4'] >= df['Q4_target'], 'Q4_rebate_achieved'] = 'YES'
df.loc[df['dollars_2021_Q4'] < df['Q4_target'], 'Q4_rebate_achieved'] = 'NO'

# Calculate Max Rebate Opportunity
df.loc[df['color'] == 'Purple', 'max_rebate_value'] = df['est_Q4_rebate_value']

# Set Q4 unit delta to integer column
df['Q4_units_needed'] = df['Q4_units_needed'].astype(int)


### Annual Rebate Columns

In [ ]:
# Set Annual Target Columns
df['cytd'] = (df['dollars_2021_Q1'] +
              df['dollars_2021_Q2'] +
              df['dollars_2021_Q3'] +
              df['dollars_2021_Q4'])

df.loc[df['base_year'] == '2019', 'annual_target'] = (df['dollars_2019_Q1'] +
                                                      df['dollars_2019_Q2'] +
                                                      df['dollars_2019_Q3'] +
                                                      df['dollars_2019_Q4']
                                                     ) * annual_target + 1


df.loc[df['base_year'] == '2020', 'annual_target'] = (df['dollars_2020_Q1'] +
                                                      df['dollars_2020_Q2'] +
                                                      df['dollars_2020_Q3'] +
                                                      df['dollars_2020_Q4']
                                                     ) * annual_target + 1

# Set rules for annual target
df.loc[df['annual_target'] < 10000, 'annual_target'] = 10000
df.loc[df['color'] == 'Purple', ['annual_target', 'annual_target_delta', 'annual_units_needed']] = 0

# Calculate Deltas
df['annual_target_delta'] = (df['annual_target'] - df['cytd']).astype(int)
df['annual_units_needed'] = round((df['annual_target_delta'] / aur)) + 1

# Calculate Annual Payout
df.loc[((df['color'] == 'Green') & (df['cytd'] > df['annual_target'])), 'est_annual_rebate_value'] = (df['cytd'] * annual_rebate_percentage).astype(int)
df.loc[((df['color'] == 'Green') & (df['cytd'] <= df['annual_target'])), 'est_annual_rebate_value'] = (df['annual_target'] * annual_rebate_percentage).astype(int)

# If annual units needed < 0, set to zero
df.loc[df['annual_units_needed'] <= 0, 'annual_units_needed'] = 0
df.loc[((df['annual_target_delta'] < 80) & (df['annual_target_delta'] > 0)), 'annual_units_needed'] = 1
df.loc[df['annual_target_delta'] < 0, 'annual_target_delta'] = 0


# Calculate Max Rebate Opportunity
df.loc[df['color'] == 'Green', 'max_rebate_value'] = df['est_Q4_rebate_value'] + df['est_annual_rebate_value']

# Determine if annual rebate has been achieved
df.loc[df['cytd'] >= df['annual_target'], 'annual_rebate_achieved'] = 'YES'
df.loc[df['cytd'] < df['annual_target'], 'annual_rebate_achieved'] = 'NO'
df.loc[df['color'] == 'Purple', 'annual_rebate_achieved'] = 'NOT ELIGIBLE'

# Set annual unit delta column to integer
df['annual_units_needed'] = df['annual_units_needed'].astype(int)

#df

In [ ]:
df.columns

### Run a test on any account number

In [ ]:
# test_account = input()
# test_account = int(test_account)
# test = df[df['code'] == test_account].copy()
# test = test[['code', 'customer', 'city', 'contact', 'email', 'color', 'base_year',
#              'dollars_2021_Q4', 'Q4_target', 'Q4_target_delta', 'Q4_units_needed', 
#              'est_Q4_rebate_value','Q4_rebate_achieved', 'cytd', 'annual_target',
#              'annual_target_delta', 'annual_units_needed', 'est_annual_rebate_value',
#             'annual_rebate_achieved', 'max_rebate_value']]

# test

### Split DataSet into "Purple" and "Green" 

In [ ]:
df.columns

In [ ]:
green = df[df['color'] == 'Green'].copy()
purple = df[df['color'] == 'Purple'].copy()

green = green[['code', 'customer', 'city', 'contact', 
               'email', 'color', 'base_year', 'dollars_2021_Q4', 'Q4_target',
               'Q4_target_delta', 'Q4_units_needed', 'est_Q4_rebate_value',
               'Q4_rebate_achieved', 'cytd', 'annual_target', 'annual_target_delta',
               'annual_units_needed', 'est_annual_rebate_value',
               'annual_rebate_achieved', 'max_rebate_value']]

purple = purple[['code', 'customer', 'city', 'contact', 
                 'email', 'color', 'base_year', 'dollars_2021_Q4', 'Q4_target',
                 'Q4_target_delta', 'Q4_units_needed', 'est_Q4_rebate_value',
                 'Q4_rebate_achieved', 'max_rebate_value']]

### Percentage of Accounts Who have Hit Targets

In [ ]:
perc_Q4_rebate_achieved = green['Q4_rebate_achieved'].value_counts(normalize = True) * 100
print("Percentage of 'Green' Accounts who have reached the Q4 Target")
print(perc_Q4_rebate_achieved)
print()
print()
perc_annual_achieved = green['annual_rebate_achieved'].value_counts(normalize = True) * 100
print("Percentage of 'Green' Accounts who have reached the Annual Target")
print(perc_annual_achieved)

In [ ]:
perc_purple_Q4_rebate_achieved = purple['Q4_rebate_achieved'].value_counts(normalize = True) * 100
print("Percentage of 'Purple' Accounts who have reached the Q4 Target")
print(perc_purple_Q4_rebate_achieved)

## Split into 4 "GREEN" Categories

In [ ]:
green_Q4_yes_annual_yes = green[(green['Q4_rebate_achieved'] == 'YES') & (green['annual_rebate_achieved'] == 'YES')]
green_Q4_yes_annual_yes = green_Q4_yes_annual_yes.sort_values("max_rebate_value", ascending = False)
green_Q4_yes_annual_yes.reset_index(drop=True, inplace=True)

if print_reports == "YES":
    green_Q4_yes_annual_yes.to_excel(f'/Users/matt/Desktop/{rep}_green_Q4_yes_annual_yes.xlsx', engine='xlsxwriter')

green_Q4_yes_annual_yes

In [ ]:
green_Q4_yes_annual_no = green[(green['Q4_rebate_achieved'] == 'YES') & (green['annual_rebate_achieved'] == 'NO')]
green_Q4_yes_annual_no = green_Q4_yes_annual_no.sort_values('annual_units_needed')
green_Q4_yes_annual_no.reset_index(drop=True, inplace=True)

if print_reports == "YES":
    green_Q4_yes_annual_no.to_excel(f'/Users/matt/Desktop/{rep}_green_Q4_yes_annual_no.xlsx', engine='xlsxwriter')

green_Q4_yes_annual_no

In [ ]:
green_Q4_no_annual_yes = green[(green['Q4_rebate_achieved'] == 'NO') & (green['annual_rebate_achieved'] == 'YES')]
green_Q4_no_annual_yes = green_Q4_no_annual_yes.sort_values("Q4_units_needed")
green_Q4_no_annual_yes.reset_index(drop=True, inplace=True)

if print_reports == "YES":
    green_Q4_no_annual_yes.to_excel(f'/Users/matt/Desktop/{rep}_green_Q4_no_annual_yes.xlsx', engine='xlsxwriter')

green_Q4_no_annual_yes

In [ ]:
green_Q4_no_annual_no = green[(green['Q4_rebate_achieved'] == 'NO') & (green['annual_rebate_achieved'] == 'NO')]
green_Q4_no_annual_no = green_Q4_no_annual_no.sort_values("annual_units_needed")
green_Q4_no_annual_no.reset_index(drop=True, inplace=True)

if print_reports == "YES":
    green_Q4_no_annual_no.to_excel(f'/Users/matt/Desktop/{rep}_green_Q4_no_annual_no.xlsx', engine='xlsxwriter')

green_Q4_no_annual_no

### Split into 2 Purple Categories

In [ ]:
purple_Q4_yes = purple[purple['Q4_rebate_achieved'] == 'YES']
purple_Q4_yes = purple_Q4_yes.sort_values("est_Q4_rebate_value", ascending = False)
purple_Q4_yes.reset_index(drop=True, inplace=True)

if print_reports == "YES":
    purple_Q4_yes.to_excel(f'/Users/matt/Desktop/{rep}_purple_Q4_yes.xlsx', engine='xlsxwriter')

purple_Q4_yes

In [ ]:
purple_Q4_no = purple[purple['Q4_rebate_achieved'] == 'NO']
purple_Q4_no = purple_Q4_no.sort_values("Q4_units_needed")
purple_Q4_no.reset_index(drop=True, inplace=True)

if print_reports == "YES":
    purple_Q4_no.to_excel(f'/Users/matt/Desktop/{rep}_purple_Q4_no.xlsx', engine='xlsxwriter')

purple_Q4_no

In [ ]:
df.columns

## Perform System Check

##### * Check You have the correct rep and contact information
##### * Check "Subject Line" in all communications
##### * Check "Message" for all communications
##### * Check data is correctly formatted

In [ ]:
system_check = input()
system_check = str(system_check)

In [ ]:
# Type YES to proceed, or NO to STOP

if system_check == 'NO':
    raise SystemExit("Don't Continue")
print('Continue')

In [ ]:
import smtplib
import config
import time

from datetime import date

today = date.today()
print(today)

## Connect to Gmail Server

In [ ]:
# def send_email(subject, msg, customer_email):
#     try:
#         server = smtplib.SMTP('smtp.gmail.com:587')
#         server.ehlo()
#         server.starttls()
#         server.login(config.EMAIL_ADDRESS, config.PASSWORD)
#         message = 'Subject: {}\n\n{}'.format(subject, msg)
#         server.sendmail(config.EMAIL_ADDRESS, customer_email, message)
#         server.quit()
#         print(f'Success. Email Sent to {customer_email}')
#         time.sleep(1)
#     except:
#         print(f'Email Failed to Send to {customer_email}')